In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u 
from astropy import constants as const
from scipy.interpolate import InterpolatedUnivariateSpline
import scipy
from scipy import integrate
from scipy.integrate import quad


In [2]:
def cross_section(E):
    L = np.log(E/1e3)
    E_th = 1.22
    return (34.3+1.88*L+0.25*pow(L,2))*pow((1-pow((E_th/E),4)),2)

In [3]:
def E_CR(RSN): #RSN in yr^-1
    E_SN_erg = 1e51*u.erg #characteristic energy output
    E_SN_GeV = (E_SN_erg.to(u.eV)).value*1e-9 # GeV
    xi = 0.10 # efficiency ~10%
    R_sn_yr = RSN*pow(u.yr,-1) # SN rate in yrs
    R_sn_s = (R_sn_yr.to(pow(u.second,-1))).value
    return R_sn_s*E_SN_GeV*xi

In [4]:
def I(p,alpha,p_max):
    mp = 0.938
    return (4*np.pi)*pow(p,2)*pow(p/mp,-alpha)*(np.sqrt(pow(p,2)+pow(0.938,2))-0.938)*np.exp(-p/p_max)



In [5]:
def Qp(p,R,h,plow,pup,alpha,pmax,RSN):
    mp = 0.938
    R_pc = R*u.parsec  #pc to cm conversion
    R_cm = (R_pc.to(u.cm)).value
    h_pc = h*u.pc
    h_cm = (h_pc.to(u.cm)).value
    if h == 0: 
        V_spherical = (4/3)*np.pi*pow(R_cm,3)
        V_SBN = V_spherical
    else:
        V_disk = 2*h_cm*np.pi*pow(R_cm,2)
        V_SBN = V_disk
        
    mom = np.logspace(np.log10(plow),np.log10(pup),100000)
    Int = np.trapz(I(mom,alpha,pmax),mom)
    N = E_CR(RSN)/Int
    return (N/V_SBN)*pow(p/mp,-alpha)*np.exp(-p/pmax)


In [1]:
def loss_time(E,nism): 
    eta = 0.5
    n_cm = nism*pow(u.cm,-3)
    n_m = (n_cm.to(pow(u.m,-3))).value
    
    sigma = cross_section(E)*1e-31
    return 1/(eta*n_m*sigma*const.c.value)



In [7]:
def tau_wind(R,v,h):
    if h==0:
        R_pc = R*u.parsec 
        R_SBN = (R_pc.to(u.m)).value
        v_wind = v*1000 
        return (R_SBN/v_wind) 
    else:
        h_pc = h*u.parsec
        h_m = (h_pc.to(u.m)).value
        v_wind = v*1000 
        return (h_m/v_wind) 
            



In [1]:
def larmor(E,B): #Larmor radius in m 
    B_G = (B*u.G)*1e-6 #Gauss
    B_T = (B_G.to(u.T)).value #Tesla
    return 3.3 *((E*1)/(1*B_T))

def W_0(k_0,d):
    integral = lambda k,d: pow(k,-d)
    I = quad(integral, k_0, np.inf, args = (d))[0] #returns list with integral and uncertainty
    return pow(pow(k_0,d)*I,-1)

def F(k,k_0,d):
    return k*W_0(k_0,d)*pow(k/k_0,-d)

def D(E,k_0,B,d):
    c = (const.c).value # lightspeed in m/s
    
    k_m = 1/(larmor(E,B)*u.m) # in m^-1
    
    k_pc = (k_m.to(pow(u.parsec,-1))).value # in pc^-1
    
    D_m2_s = ((larmor(E,B)*c)/(3*F(k_pc,k_0,d)))*(pow(u.m,2)/u.s) #in m^2/s
    
    D_pc2_s = (D_m2_s.to(pow(u.pc,2)/u.s)).value
    
    return D_pc2_s # in pc^2/s

def tau_diff_quasi(E,R):
    return pow(R,2)/D(E,1,250,5/3)



In [9]:
def tau_lifetime(R,vwind,E , nism,h):
    return pow(pow(tau_wind(R,vwind,h),-1)+pow(loss_time(E,nism),-1) +pow(tau_diff_quasi(E,R),-1),-1)


In [10]:
def f_p(E,R,v,nism,h,plow,pup,alpha,pmax,RSN):
    return tau_lifetime(R,v, E, nism,h)*Qp(E,R,h,plow,pup,alpha,pmax,RSN)

In [1]:
def q_pi(E,nism, R,h,v,plow,pup,alpha,pmax,RSN):
    n = nism*1e6 
    c = 3e8    
    K_pi = 0.17 # kinetic energy transfer parent proton to pion 
    m_p = 0.938 # GeV
    E_proton = E/(K_pi)+0.938
    return (c*n)*(1/K_pi)*(cross_section(E_proton)*1e-31)*4*np.pi*(pow(E_proton,2)-pow(0.938,2))*tau_lifetime(R,v, E_proton, nism,h)*Qp(E_proton,R,h,plow,pup,alpha,pmax,RSN)




In [12]:
def g_mu(x):
    r = 0.573
    return (9*pow(x,2)-6*np.log(x)-4*pow(x,3)-5)*(3-2*r)/(9*pow(1-r,2))

def h_mu1(x):
    r = 0.573
    return (9*pow(r,2)-6*np.log(r)-4*pow(r,3)-5)*(3-2*r)/(9*pow(1-r,2))

def h_mu2(x):
    r = 0.573
    return ((9*(r+x)-4*(pow(r,2)+r*x+pow(x,2)))*(1+2*r)*(r-x))/(9*pow(r,2))



def ge(x):
    r = 0.573
    return 2*(((1-x)*(6*(1-x)**2+r*(5+5*x-4*x**2))+6*r*np.log(x)))/(3*(1-r)**2)


def h_e1(x):
    r = 0.573
    return (2/(3*(pow((1-r),2))))*((1-r)*(6-7*r+11*pow(r,2)-4*pow(r,3))+6*r*np.log(r))

def h_e2(x):
    r = 0.573
    return((2*(r-x))/(3*pow(r,2))) *(7*pow(r,2)-4*pow(r,3)+7*x*r-4*x*pow(r,2)-2*pow(x,2)-4*pow(x,2)*r)

In [13]:
def f_mu1(x):
    lambd = 1-0.573
    if (lambd-x>=0):
        return 1/lambd
    else: 
        return 0

def f_mu2(x):
    r = 0.573
    if (x-r>=0):
        return g_mu(x)
    if (r-x>=0):
        return h_mu1(x)+h_mu2(x)
        
    else:
        return 0

def f_ve(x):
    r =0.573
    if (x - r >= 0):
        return ge(x)
    if (r-x > 0):
        return h_e1(x)+h_e2(x)
    else:
        return 0

In [14]:
def q_nu(x,E_nu, nism, R,h,v,plow,pup,alpha,pmax,RSN):
    return (2/3)*(f_mu1(x)+f_mu2(x)+f_ve(x))*q_pi(E_nu/x,nism, R,h,v,plow,pup,alpha,pmax,RSN)*(1/x)


In [15]:
def bulk_flux(E,nism,R,h,v,plow,pup,alpha,pmax,RSN,D_L):
    DL_pc = (D_L*1e6)*u.parsec 
    DL_cm = (DL_pc.to(u.cm)).value
    R_pc = R*u.pc
    R_cm = (R_pc.to(u.cm)).value
    h_pc = h*u.parsec
    h_cm = (h_pc.to(u.cm)).value

    
    I = quad(q_nu,0,1,args = (E,nism, R,h,v,plow,pup,alpha,pmax,RSN) )[0] #takes factor three already into account
    fluxes = I
    
    if h!= 0: 
        V = 2*h_cm*np.pi*pow(R_cm , 2 )
    else: 
        V = (4/3)*np.pi*pow(R_cm,3)
    
    scaled_flux = (V/(4*np.pi*pow(DL_cm,2)))*pow(E,2)*fluxes
    
    return [scaled_flux]